In [45]:
import sqlite3 as lite
from monitoring import DB_PATH, SENSORS, SensorDB



In [5]:

import pandas as pd

# scp axlan@192.168.1.110:~/src/cat_chair/controller/*.sqlite controller/
con = lite.connect(f'file:{DB_PATH}?mode=ro', uri=True)
df = pd.read_sql_query("SELECT * from sensor_data", con, parse_dates=['timestamp'])
con.close()

In [48]:
import plotly.express as px

df_disp = df[df['value'] >= 0]
df_disp['timestamp'] = df_disp['timestamp'].dt.tz_localize(tz='UTC')
df_disp['timestamp'] = df_disp['timestamp'].dt.tz_convert(tz='US/Pacific')

#
# print(df)

fig = px.scatter(df_disp, x="timestamp", y="value", color="name")
fig.show()